In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

#Import PySwarms
import pyswarms as ps

# Load the iris dataset
data = load_iris()

#Store the features as X and the label as y
X = data.data
y = data.target

def forward_prop(params):

    #Neural network architecture
    n_inputs = 4
    n_hidden = 20
    n_classes = 3

    # Roll-back the weights and biases
    W1 = params[0:80].reshape((n_inputs,n_hidden))
    b1 = params[80:100].reshape((n_hidden,))
    W2 = params[100:160].reshape((n_hidden,n_classes))
    b2 = params[160:163].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1 # Pre-activation in Layer 1
    a1 = np.tanh(z1) # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2 # Logits for Layer 2

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood
    N = 150 # Number of samples
    corect_logprobs = -np.log(probs[range(N), y])
    loss = np.sum(corect_logprobs) / N
    return loss

def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.
    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
    The swarm that will perform the search
    Returns
    -------
    numpy.ndarray of shape (n_particles, )
    The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]

    return np.array(j)

# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
# Call instance of PSO
dimensions = (4 * 20) + (20 * 3) + 20 + 3
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions,
options=options)
# Perform optimization
cost, pos = optimizer.optimize(f, iters=1000)

def predict(X, pos):
    """
    Use the trained weights to perform class predictions.
    Inputs
    ------
    X: numpy.ndarray
    Input Iris dataset
    pos: numpy.ndarray
    Position matrix found by the swarm. Will be rolled
    into weights and biases.
    """
    # Neural network architecture
    n_inputs = 4
    n_hidden = 20
    n_classes = 3
    # Roll-back the weights and biases
    W1 = pos[0:80].reshape((n_inputs,n_hidden))
    b1 = pos[80:100].reshape((n_hidden,))
    W2 = pos[100:160].reshape((n_hidden,n_classes))
    b2 = pos[160:163].reshape((n_classes,))
    # Perform forward propagation
    
    z1 = X.dot(W1) + b1 # Pre-activation in Layer 1
    a1 = np.tanh(z1) # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2 # Logits for Layer 2
    y_pred = np.argmax(logits, axis=1)
    return y_pred

acc=(predict(X, pos) == y).mean()
print(acc)

2022-11-29 16:12:04,341 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0.031
2022-11-29 16:12:17,066 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.031015435826767596, best pos: [ 5.39429763e-01 -6.95472801e-02  2.90325055e-01 -1.27308812e+00
 -2.87007446e+00  6.39469264e-01  1.14739205e+00 -5.88081559e-01
 -1.04234793e+00  3.65987901e+00  4.68306819e-01  1.46464893e+00
  4.80501582e-01 -7.20829760e-01  2.56206960e+00 -1.28790101e+00
  4.47859826e-01  2.53106748e+00  6.35794523e-01 -3.56839969e+00
 -1.72341309e+00 -5.86088349e-01  4.08514562e-01  2.22244037e-01
  9.95078509e-01  5.22154996e-01 -1.21593974e+00 -4.65036014e-01
 -6.37511253e-01  2.17560709e-02  2.83954113e+00 -6.88249786e-01
  8.47390600e-01  1.81843394e+00  2.42330538e-01  1.92091171e-01
  1.15357378e+00  8.61738437e-01  5.24035138e-01 -2.87607111e+00
  1.09770387e+00  1.420